# dictionary와 tuple보다는 핼퍼 클래스로 관리하자

Python에 내장되어 있는 dictionary 타입은 객체의 수명이 지속되는 동안 동적인 내부 상태를 관리하는 용도로 좋다.  
__'동적'이란 예상하지 못한 식별자들을 관리해야 하는 상황을 뜻한다.__

이름을 모르는 학생 집단의 성적을 기록하는 클래스

In [1]:
class SimpleGradebook(object):
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = []
        
    def report_grade(self, name, score):
        self._grades[name].append(score)
        
    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)

클래스의 사용법은 간단하다.

In [3]:
book = SimpleGradebook()
book.add_student('Isaac Newton')
book.report_grade('Isaac Newton', 90)
# ...
print(book.average_grade('Isaac Newton'))

90.0


In [ ]:
dictionary는 정말 사용하기 쉬워서 과도하게 쓰다가 코드를 취약하게 작성할 위험이 있다.